In [21]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

# from keras.models import following models, layers  
from keras.models import Sequential
from keras.layers import InputLayer, Input
from keras.layers import Reshape, MaxPooling2D
from keras.layers import Conv2D, Dense, Flatten
from keras.optimizers import Adam

In [22]:
# Loading data
# We have used "One-hot encoding" -- in which a given label is converted from single number to
# vector whose length equals the number of possible classes.

# For eg: If given label is of an image charater "7" -- It's corresponding vector would be having 
# 10 elements, with all elements be 0 and only 7th element be 1.
# Ref: https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [23]:
# Keep the test lables (classes) as single numbers as well for various comparison and performance measures.
# So we convert the One-Hot encoded vectors back to a single number by taking the index of the highest element. 

mnist.test.cls = np.array([label.argmax() for label in mnist.test.labels])

In [24]:
# Setting up the CNN
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

In [25]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
# This is used for plotting the images.
img_shape = (img_size, img_size)

# Tuple with height, width and depth used to reshape arrays.
# This is used for reshaping in Keras.
img_shape_full = (img_size, img_size, 1)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 10

In [26]:
# Sequitial Models in Keras

# Start construction of the Keras Sequential model.
model = Sequential()

# Add an input layer which is similar to a feed_dict in TensorFlow.
# Note that the input-shape must be a tuple containing the image-size.
model.add(InputLayer(input_shape=(img_size_flat,)))

# The input is a flattened array with 784 elements,
# but the convolutional layers expect images with shape (28, 28, 1)
model.add(Reshape(img_shape_full))

# First convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=5, strides=1, filters=16, padding='same',
                 activation='relu', name='layer_conv1'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Second convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=5, strides=1, filters=36, padding='same',
                 activation='relu', name='layer_conv2'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Flatten the 4-rank output of the convolutional layers
# to 2-rank that can be input to a fully-connected / dense layer.
model.add(Flatten())

# First fully-connected / dense layer with ReLU-activation.
model.add(Dense(128, activation='relu'))

# Last fully-connected / dense layer with softmax-activation
# for use in classification.
model.add(Dense(num_classes, activation='softmax'))


In [27]:
# Model Compilation
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
# Training

model.fit(x=mnist.train.images,
          y=mnist.train.labels,
          epochs=5, batch_size=128)

Epoch 1/5
55000/55000 [==============================] - 51s 934us/step - loss: 0.2119 - acc: 0.9377
Epoch 2/5
55000/55000 [==============================] - 50s 916us/step - loss: 0.0612 - acc: 0.9808
Epoch 3/5
55000/55000 [==============================] - 50s 917us/step - loss: 0.0401 - acc: 0.9871
Epoch 4/5
55000/55000 [==============================] - 61s 1ms/step - loss: 0.0309 - acc: 0.9904
Epoch 5/5
55000/55000 [==============================] - 65s 1ms/step - loss: 0.0249 - acc: 0.9920


In [29]:
# Evaluation

result = model.evaluate(x=mnist.test.images,
                        y=mnist.test.labels)

for name, value in zip(model.metrics_names, result):
    print(name, value)
    
    
print("{0}: {1:.2%}".format(model.metrics_names[1], result[1]))

10000/10000 [==============================] - 6s 555us/step
loss 0.03268984900137875
acc 0.9884
acc: 98.84%


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 784)               0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
layer_conv1 (Conv2D)         (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
layer_conv2 (Conv2D)         (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1764)              0         
__________